In [2]:
import pandas as pd
import numpy as np
import re
from transformers import BertTokenizer, BertForMaskedLM
import torch
from transformers import AdamW
from tqdm import tqdm
import gensim
from gensim.models import Word2Vec 
from gensim.models import KeyedVectors
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import pickle
import ast
import statistics as s
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
import nltk
from transformers import pipeline
import copy
from nltk.corpus import stopwords

In [3]:
tokenizer = BertTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased")
#BERT_model = BertForMaskedLM.from_pretrained("GroNLP/bert-base-dutch-cased")
word2vec_model = KeyedVectors.load_word2vec_format("combined-320.txt")
# https://github.com/clips/dutchembeddings

#df = pd.read_csv('preprocessed_df.csv')

In [5]:
# list of unique words w2v model original
vocab_word2vec = list(word2vec_model.vocab.keys())
vocab_word2vec = list(set(vocab_word2vec))
print('org vocab word2vec count:', len(vocab_word2vec))
# list of unique words BERT model original
vocab_BERT = list(tokenizer.vocab.keys())
vocab_BERT = list(set(vocab_BERT))
print('org vocab BERT count:', len(vocab_BERT))
# list of unique words finetuned on
with open('gtfortraining.txt', 'rb') as f:
    train = pickle.load(f)
train = list(set(train))
print('finetuned unique words:', len(train))
# merge to create new vocabularies
vocab_word2vec = vocab_word2vec + train
vocab_word2vec = list(set(vocab_word2vec))
print('pretrained and finetuned vocab combined w2v:', len(vocab_word2vec))
vocab_BERT = vocab_BERT + train
vocab_BERT = list(set(vocab_BERT))
print('pretrained and finetuned vocab combined BERT:', len(vocab_BERT))

org vocab word2vec count: 1442950
org vocab BERT count: 30073
finetuned unique words: 131823
pretrained and finetuned vocab combined w2v: 1510565
pretrained and finetuned vocab combined BERT: 151245


30073
1442950
67231
1442950


In [6]:
# create list of historical expressions
with open('dictionaryBasedDutchDictionary_1.0.type_frequency.txt', 'r', encoding="utf8") as f: # part 1 historical expressions
    words = f.readlines()
hist_words1 = [word[0 : word.index("\t")] for word in words]
#print(woorden)

with open('dutchCorpusBasedDictionary_1.1.tf.txt', 'r', encoding="utf8") as f: # part 2 historical expressions
    words = f.readlines()
hist_words2 = [word[0 : word.index("\t")] for word in words]
#print(woorden)

expressions = list(set(hist_words1).union(set(hist_words2))) # all historical expressions

with open('wordlist.txt', 'r', encoding="utf8") as f: # modern_vocab
    words = f.read().split('\n')
    #words3 = [word[0 : word.index("\n")] for word in words]
    modern_vocab = words
    
# historical expressions is everything in hist_expression that is not in modern_vocab
hist_expressions = list(set(expressions).difference(set(modern_vocab)))
print(hist_expressions[:100])
print(len(hist_expressions))
print(len(modern_vocab))

['asphaltbereider', 'Luitenant-Colonel', 'koortsachtigheid', 'verdaadighschrift', 'bewooneren', 'Schaker', 'gemakkajemd', 'Paisiblement', 'verstocket', 'doorsteekend', 'offerpenningen', 'zuijelijken', 'kluchtjes', 'tijdboeken', 'stekelhage', 'voorzangster', 'November', 'steppenwolf', 'Vallot', 'kugler', 'azimuthcirkels', 'oorsaecks', 'Musique', 'sla-bek', 'Abendlied', 'uitbengelen', 'irréductible', "p'ruyck", 'Geertruyd', 'tsire', 'gumins', 'Hingman', 'adiutorio', 'opper-bestuur', 'onwedersprekelijke', 'eyser', 'distraheren', 'wormlyders', 'twantelt', 'wegh-gesellen', 'kettingseizingen', 'zaadvochtachtig', 'scheefgeloopen', 'kegellaag', '837', "t'vierige", 'zengere', 'reeckenaers', 'interficiendi', 'apologia', 'uitwaazemen', 'patroonen', 'Westph', 'ontzettendste', 'veritable', 'accusationem', 'Funktionen', 'standsgelegenheid', 'zooghe', 'beringhe', 'wond-doorn', 'geloovich', 'Beschimpte', 'uytgespreydt', 'Costelijck', 'waareren', 'damassé', 'poigneerdeert', 'tooneelkapper', 'Apollodoru

In [7]:
print('aen' in hist_expressions)
print('aan' in modern_vocab)
print('aan' in hist_expressions)
print('aen' in modern_vocab)

True
True
False
False


In [8]:
# create list with homonyms
with open('homoniemen.txt', 'r', encoding="utf8") as f: # part 1 historical expressions
    homonyms = f.readlines()
homonyms = [word[0 : word.index(":")] for word in homonyms]
print(homonyms[0:10])

['horen', 'vuil', 'vizier', 'militant', 'naturel', 'primetime', 'koperdraad', 'tel', 'zondagmiddag', 'plastiek']


In [10]:
# create list with infrequent expressions
# = domain expressions etc. Not in original vocab, but a few times in the new training set.
#model_vocabs = list(tokenizer.vocab.keys()) + list(model.vocab.keys())
# find infrequent expressions
# merge all GT text to list of words
# count unique words
import re
from collections import Counter
occurrences = Counter(word for word in train)
occurrences_counts = Counter(occ for occ in occurrences.values())
infrequent_expressions = [key for key , value in occurrences.items() if value == 1]

In [11]:
def process(tokens_list):
    tokens_list = [re.sub(r'[^\w\d\s\-]+', '', x) for x in tokens_list]
    tokens_list = [x.lower() for x in tokens_list]
    tokens_list = [re.sub(r'[^a-z]+', '', x) for x in tokens_list]
    tokens_list = list(filter(None, tokens_list))
    return tokens_list

# make sure all lists are in lowercase and without punctuation except for hyphens
# homonyms
homonyms = process(homonyms)
# BERTs vocab:
vocab_BERT = process(vocab_BERT)
# word2vecs vocab:
vocab_word2vec = process(vocab_word2vec)
# historical expressions
hist_expressions = process(hist_expressions)
# the modern vocabulary
modern_vocab = process(modern_vocab)
# infrequent expressions
infrequent_expressions = process(infrequent_expressions)
# the dictionary for the baseline model
dictionary = process(modern_vocab + hist_expressions)

In [12]:
# save al lists and later unpack:
#all_lists_tokens = [homonyms, vocab_BERT, vocab_word2vec, hist_expressions, modern_vocab, infrequent_expressions, dictionary]
#all_lists_tokens = [homonyms, hist_expressions, modern_vocab, infrequent_expressions, dictionary]

with open('homonyms.txt', 'wb') as f:
    pickle.dump(homonyms, f)
with open('vocab_BERT.txt', 'wb') as f:
    pickle.dump(vocab_BERT, f)
with open('vocab_word2vec.txt', 'wb') as f:
    pickle.dump(vocab_word2vec, f)
with open('hist_expressions.txt', 'wb') as f:
    pickle.dump(hist_expressions, f)
with open('infrequent_expressions.txt', 'wb') as f:
    pickle.dump(infrequent_expressions, f)
with open('dictionary.txt', 'wb') as f:
    pickle.dump(dictionary, f)




#with open('all_lists_tokens.txt', 'wb') as f:
#    pickle.dump(all_lists_tokens, f)